# "EarthAnnotator"

## Prototype web-based image labeling tool

Daniel Buscombe, daniel.buscombe@nau.edu October 2018
Mods. by Chris Sherwood, csherwood@usgs.gov March 2019

Implements the technique outlined by [Buscombe & Ritchie (2018)](https://www.mdpi.com/2076-3263/8/7/244)
This version writes the same output info as the version in dl_tools.

### 1. Load libraries

In [ ]:
from PIL import Image ##from imageio import imread
import numpy as np
import matplotlib.pyplot as plt
import holoviews as hv
from holoviews.streams import FreehandDraw
hv.extension('bokeh')

In [ ]:
from funcs.crf_utils import *
from funcs.widgets_utils import *
from funcs.file_select import FileBrowser
from funcs.tile_utils import sliding_window
global labels_widget

In [ ]:
import panel, param
from bokeh.plotting import figure
from panel.layout import *
from panel.widgets import *
panel.extension()
from joblib import Parallel, delayed, cpu_count
from scipy.io import savemat

### 2. Pick an image 

#### 2a. Use the GUI 

In [ ]:
file_picker = FileBrowser()
file_picker.widget()
imfile = file_picker.path

#### 2b. Or hardwire the input image

In [ ]:
# imfile = '/home/jovyan/EarthAnnotator/data/NewRiver_worldImageryRGB_20m.tif'

In [ ]:
image_path = os.path.normpath(imfile)

In [ ]:
## TODO - why do we need PIL to open the image?
im = Image.open(image_path)
im = im.convert("RGB")
nx, ny, nz = np.shape(im)
print('Image is',image_path,'with size',nx,'x',ny,'x',nz)

### 3. Make labels and colors

#### 3a. Use the GUI
#### Instructions:
    Create class labels and assign each label a color

In [ ]:
label_editor, labels_widget = create_colorpicker()
label_editor

In [ ]:
labels, colors = get_labels_and_colors(label_editor)

#### 3b. Or hardwire the labels and colors

In [ ]:
# Matanzas labels and colors
labels = ['sand','wetland_veg','water','dune_grass', 'woody_veg', 'structure','road','surf']
colors = ['#FEE893', '#5F7D8E', '#0052A5', '#8DD080', '#076443', '#868e96','#808080','#0000ff']

In [ ]:
labels

In [ ]:
colors

### 4. Freehand Drawing

#### Instructions:
1. Choose a label from the dropdown menu (and optionally choose a line width)
2. Click the 'pen' tool, and freehand draw on image. If you screw up, tap a line to select it then press BACKSPACE key while the mouse is within the plot area.
3. When done with freehand draw for each label, click "Done with Label"
4. Choose next label and repeat steps 1-3.
5. When done with all labels, proceed to next cell

In [ ]:
def set_active_tool(plot, element):
    """set freehand draw tool to be initially active"""
    plot.state.toolbar.active_drag = plot.state.tools[5]

class EarthAnnotate(param.Parameterized):
    anno = {}
    label = param.ObjectSelector(objects=labels, default=labels[0]) 
    done_with_label = param.Action(lambda x: x.anno.update( **{x.label: 
                            np.column_stack(access_annotation_coordinates(x.freehand_stream))}),
                                 precedence=1.0)
    
    def make_view(self, **kwargs):
        color_index = labels.index(self.label)
        opts = dict(line_width=brush, color=colors[color_index], 
                    finalize_hooks=[set_active_tool], width=w, height=h)
        path = hv.Path([[(0, 0), (0, 0)]]).options(**opts)
        self.freehand_stream = FreehandDraw(source=path, num_objects=999)
        bounds=(0,0,nyt,nxt)   # Coordinate system: (left, bottom, top, right)
        img = hv.RGB(imt, bounds=bounds) #hv.Image(im, bounds=bounds)

        return img * path

In [ ]:
# all these variables in the workspace end up **kwargs ??
# tile size for input image...but in this version, only one tile
nyt = 650
nxt = 650
imt=np.array(im)
w=nxt
h=nyt
# intial brush size
brush=3
e_anno = EarthAnnotate(name="Image Annotation")
panel.Row(e_anno.make_view, e_anno)

In [ ]:
anno = e_anno.anno

### 5. Set parameters

In [ ]:
# Specify the size of the tiles (96 is smallest)
tile = 96 

#### Make directories for example tiles

In [ ]:
outpath = 'tiles'+str(tile)
#=======================================================
try:
  os.mkdir(outpath)
except:
  pass

for f in labels:
  try:
     os.mkdir(outpath+os.sep+f)
  except:
     pass

In [ ]:
# This parameter determines what fraction of the tile must match the label
thres = .9 # fraction of tile that must match label

In [ ]:
# This parameter penalizes small pieces of segmentation that are
# spatially isolated -- enforces more spatially consistent segmentations
compat_spat=12 ##non-dimensional
# larger values = larger pieces of segmentation allowed

In [ ]:
# This parameter penalizes pieces of segmentation that are
# less uniform in color -- enforces more consistent segmentations in colorspace
compat_col=40 # value from int_seg_crf_matanzas.py
#compat_col=100 #non-dimensional
# larger values = pieces of segmentation with less similar image intesity allowed

In [ ]:
# Scaling parameters: tolerances in intensity and location
# theta = 100 # value from int_seg_crf_matanzas.py
theta=60 
# larger values = pixel pairs can be considered to be the same class label with less similar location/intensity

In [ ]:
# number of iterations for algorithm (generally, larger the better, but only to a point)
# n_iter=15
n_iter=20 # value from int_seg_crf_matanzas.py

In [ ]:
### 6. Run pixel model

In [ ]:
%%time
Lc = get_sparse_label(anno, nx, ny, labels, brush)
res , p= getCRF(im, Lc, theta, n_iter, labels, compat_spat, compat_col)

In [ ]:
out = get_rgb(res, labels, colors)

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(im)
plt.imshow(out, alpha=0.5);

### 7. Write image to file

In [ ]:
write_label_image(imfile, out)

In [ ]:

savemat(image_path.split('.')[0]+'_mres.mat', {'sparse': Lc.astype('int'), 'class': res.astype('int'), 'preds': p.astype('float16'), 'labels': labels}, do_compression = True)


In [ ]:
print('Generating tiles from dense class map ....')

Z,ind = sliding_window(np.asarray(im), (tile,tile,3), (int(tile/2), int(tile/2),3))

C,ind = sliding_window(res, (tile,tile), (int(tile/2), int(tile/2)))

w = Parallel(n_jobs=-1, verbose=0, pre_dispatch='2 * n_jobs', max_nbytes=None)(delayed(writeout)(Z[k], C[k], labels, outpath+os.sep, thres) for k in range(len(Z)))